In [249]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds



In [250]:
import matplotlib.pyplot as plt
import pandas as pd


In [263]:
#comment data_dir first time you use this cell
data_dir = '~/tensorflow_datasets'

(train_ds, val_ds, test_ds), metadata = tfds.load(
    'plant_village',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
    data_dir=data_dir
)

In [253]:
for image, labels in train_ds.take(1): 
  print(image.shape)

(256, 256, 3)


In [254]:
num_classes = metadata.features['label'].num_classes

In [261]:

#def preprocess_data(image, label):
#  label = tf.one_hot(label, num_classes)
#  return image, label

#train_ds = train_ds.map(preprocess_data)
#val_ds = val_ds.map(preprocess_data)
#test_ds = test_ds.map(preprocess_data)



In [255]:
train_ds = train_ds.batch(32)

val_ds = val_ds.batch(32)


In [264]:
test_ds = test_ds.batch(32)

In [ ]:
metadata.features['image'].shape 

(None, None, 3)

In [ ]:
input_shape = metadata.features['image'].shape


In [257]:

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255, input_shape=(256,256,3)),
  tf.keras.layers.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.RandomRotation(0.2),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(38, activation='softmax')
])

In [258]:
model

In [259]:

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [260]:
model.fit(train_ds,
          validation_data=val_ds,
          epochs=5)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1358/1358 [==============================] - 159s 114ms/step - loss: 1.8191 - accuracy: 0.4959 - val_loss: 2.0677 - val_accuracy: 0.4721
Epoch 2/5
1358/1358 [==============================] - 150s 111ms/step - loss: 1.1001 - accuracy: 0.6671 - val_loss: 1.2437 - val_accuracy: 0.6548
Epoch 3/5
1358/1358 [==============================] - 151s 111ms/step - loss: 0.9088 - accuracy: 0.7244 - val_loss: 1.1145 - val_accuracy: 0.6791
Epoch 4/5
1358/1358 [==============================] - 152s 112ms/step - loss: 0.8107 - accuracy: 0.7527 - val_loss: 0.8423 - val_accuracy: 0.7564
Epoch 5/5
1358/1358 [==============================] - 152s 112ms/step - loss: 0.7410 - accuracy: 0.7681 - val_loss: 0.7296 - val_accuracy: 0.7779


In [265]:
model.evaluate(test_ds)

170/170 [==============================] - 4s 24ms/step - loss: 0.6729 - accuracy: 0.7910


[0.6728978157043457, 0.7909760475158691]